In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum
from scipy.integrate import quad

yu.setpath('lhapdf')

if "_has_run_once" not in globals():
    import os, sys, ctypes
    _has_run_once = True
    sys.path.insert(0, "/p/project1/ngff/li47/code/LHAPDF/build/lib/python3.13/site-packages")
    ctypes.CDLL("/p/project1/ngff/li47/code/LHAPDF/build/lib/libLHAPDF.so", mode=ctypes.RTLD_GLOBAL)
    import lhapdf

In [2]:
lhapdf.availablePDFSets()

['NNPDF31_nnlo_as_0118',
 'NNPDF40_an3lo_as_01180',
 'NNPDF40_nlo_as_01180',
 'NNPDF40_nnlo_as_01180',
 'NNPDF40_nnlo_as_01180_qed']

In [2]:
PDF_SET      = "NNPDF40_nnlo_as_01180"
pdfset = lhapdf.getPDFSet(PDF_SET)
replicas = pdfset.mkPDFs()

LHAPDF 6.5.5 loading all 101 PDFs in set NNPDF40_nnlo_as_01180
NNPDF40_nnlo_as_01180, version 1; 101 PDF members


In [3]:
# avgx
def j2inds(j):
    if j in ['ju','jd','js','jc','jg']:
        return {'ju':[2,-2], 'jd':[1,-1], 'js':[3,-3], 'jc':[4,-4], 'jg':[21]}[j]
    if j in ['jq']:
        return np.concatenate([j2inds(j) for j in ['ju','jd','js','jc']])
    if j in ['jtot']:
        return np.concatenate([j2inds(j) for j in ['jq','jg']])
    
def pdf_moment(pdf, inds, n=1):
    Q=2.0
    func = lambda x: x**(n-1) * sum(pdf.xfxQ(ind, x, Q) for ind in inds)
    val, _ = quad(func, 1e-6, 1.0, epsabs=1e-6, epsrel=1e-4)
    return val

js=['jtot','jq','jg','ju','jd','js','jc']
j2dat={}
for j in js:
    dat=[pdf_moment(pdf, j2inds(j)) for pdf in replicas]
    j2dat[j]=np.array(dat)

j2me={}
for j in js:
    dat=j2dat[j]
    m,e=dat.mean(),dat.std(ddof=1)
    j2me[j]=(m,e)
    print(j,yu.un2str(m,e))
    if j in ['jg']:
        print()
    
yu.save_pkl_reg('j2me_NNPDF40_nnlo_as_01180',j2me)

jtot 0.9999969(47)
jq 0.6005(26)
jg 0.3995(26)

ju 0.3468(26)
jd 0.1934(32)
js 0.0483(34)
jc 0.0119(22)
